# Position Player fWAR
___

           {    (Batting Runs + Base Running Runs + Fielding Runs +        }
    fWAR = {  Positional Adjustment + League Adjustment +Replacement Runs) }
           {                      / (Runs Per Win)                         }


## Dynamic inputs

In [1]:
import pandas as pd
import numpy as np
import sqlite3

import config

In [2]:
con = sqlite3.connect(config.mlb_db_path)

In [3]:
df = pd.read_sql_query("""SELECT * 
                       FROM {} 
                       WHERE (gameid like '%OAK%' OR vteam='OAK')
                       """.format('bevent'),
                       con=con)

In [4]:
play_defs = {
     0:'Unknown event',
     1:'No event',
     2:'Generic out',
     3:'Strikeout',
     4:'Stolen base',
     5:'Defensive indifference',
     6:'Caught stealing',
     7:'Pickoff error',
     8:'Pickoff',
     9:'Wild pitch',
     10:'Passed ball',
     11:'Balk',
     12:'Other advance',
     13:'Foul error',
     14:'Walk',
     15:'Intentional walk',
     16:'Hit by pitch',
     17:'Interference',
     18:'Error',
     19:'Fielders choice',
     20:'Single',
     21:'Double',
     22:'Triple',
     23:'Home run',
     24:'Missing play'}

## Batting Runs
___
    player’s:
        wOBA
        PA
        home park factor 
    League: 
        Average wOBA (lgwOBA)
        the wOBA Scale
        MLB R/PA (lgR/PA)
        Specific league (AL or NL) wRC and PA for non-pitchers
___
    >> wRAA = ((wOBA – lgwOBA)/wOBA Scale) * PA
    >>> Batting Runs = wRAA + (lgR/PA – (PF*lgR/PA))*PA + (lgR/PA – (AL or NL non-pitcher wRC/PA))*PA

In [5]:
class metric_calculator(object):
    def __init__(self, data):
        self.mts = stat_metrics(data)
        self.metric_fxns = {
            'BA':self.mts.batting_avg,
            'SLG':self.mts.slugging,
            'wOBA':self.mts.wOBA,
            'wRAA':self.mts.wRAA,
            'UZR':self.mts.UZR,
            'fWAR':self.mts.fWAR}
            
    def calculate(self, player_ids, metric):
        self.metric_exists(metric)
        metric_calc = self.metric_fxns[metric]
        value = metric_calc(player_ids)
        return value
        
    def metric_exists(self, metric):
        assert metric in self.metric_fxns.keys(), "Invalid metric, metric not defined"
        return None

In [64]:
class stat_metrics(object):
    def __init__(self, data):
        self.df = data
        self.war_position_adj = {}
        self.fg_constants = {'wOBA':.315,
                        'wOBAScale':1.226,
                        'wBB':.690,
                        'wHBP':.720,
                        'w1B':.880,
                        'w2B':1.247,
                        'w3B':1.578,
                        'wHR':2.031} 
        
    def pre_process(self, df, process_dict):
        event_specs =  pd.DataFrame(
            index=np.arange(26)
            ).join(df.groupby('eventtype').agg(process_dict)).fillna(0)
        print(event_specs)
        return event_specs
    
    def player_df(self, player_ids, stat_type='batter'):
        if type(player_ids) == str:
            player_ids = [player_id]
        elif type(player_ids) == list:
            pass
        else:
            assert "Invalid Player ID/List"
        
        return self.df[self.df[stat_type].isin(player_ids)]
            
    #####################################
    ############ BATTING  ###############
    #####################################
    def wOBA(self, player_ids):
        df = self.player_df(player_ids=player_ids, stat_type='batter')
        ## General Stats
        df_ = self.pre_process(df, {'gameid':'count'})['gameid']
        hbp = df_.loc[16]
        ibb = df_.loc[15]
        bb = df_.loc[14] + ibb
        b1 = df_.loc[20]
        b2 = df_.loc[21]
        b3 = df_.loc[22]
        hr = df_.loc[23]
        ab = len(df[df['abflag'] == 'T'])
        ibb = df_.loc[15]
        ## SF calculation
        sf = len(df[df['sfflag'] == 'T'])
        value = (
            (self.fg_constants['wBB']*(bb-ibb) + 
            self.fg_constants['wHBP']*hbp + 
            self.fg_constants['w1B']*b1 + 
            self.fg_constants['w2B']*b2 + 
            self.fg_constants['w3B']*b3 + 
            self.fg_constants['wHR']*hr)/
                (ab+bb-ibb+sf+hbp))
        return value
    
    def batting_avg(self, player_ids):
        df = self.player_df(player_ids=player_ids, stat_type='batter')
        df_ = self.pre_process(df, {'gameid':'count'})['gameid']
        b1 = df_.loc[20]
        b2 = df_.loc[21]
        b3 = df_.loc[22]
        hr = df_.loc[23]
        ## AB flag count
        ab = len(df[df['abflag'] == 'T'])
        value = (b1+b2+b3+hr)/ab
        return value

    def slugging(self, player_ids):
        df = self.player_df(player_ids=player_ids, stat_type='batter')
        df_ = self.pre_process(df, {'gameid':'count'})['gameid']
        b1 = df_.loc[20]
        b2 = df_.loc[21]
        b3 = df_.loc[22]
        hr = df_.loc[23]
        ## AB flag count
        ab = len(df[df['abflag'] == 'T'])
        value = (b1+(b2*2)+(b3*3)+(hr*4))/ab
        return value
    
    def wRAA(self, player_ids):
        df = self.player_df(player_ids=player_ids, stat_type='batter')
        wOBA = self.wOBA(player_ids=player_ids)
        df_ = self.pre_process(df, {'gameid':'count'})['gameid']['count']
        ab = len(df[df['abflag'] == 'T'])
        hbp = df_.loc[16]
        ibb = df_.loc[15]
        bb = df_.loc[14] + ibb
        sh = len(df[df['shflag'] == 'T'])
        ## SF calculation
        sf = len(df[df['sfflag'] == 'T'])
        value = ((wOBA-self.fg_constants['wOBA'])/self.fg_constants['wOBAScale'])*(ab+bb+hbp+sf+sh)
        return value
    
    def UZR(self, player_ids):
        df = self.player_df(player_ids=player_ids, stat_type='batter')
        
        return value
    
    @staticmethod
    def position_determination(player_ids):
        df = self.player_df(player_ids=player_ids, stat_type='batter')
        df[['gameid','defenseiveposition']].groupby('defensiveposition').count().sort_values('count').index.tolist()[0]
        return position
    
    def fWAR(self, player_ids):
        df = self.player_df(player_ids=player_ids, stat_type='batter')
        wRAA = self.wRAA(df)
        UZR = self.UZR(df)
        pos = self.position_determination(df)
        position = self.positon_adj[pos]
        pa = df_.sum() # Calcing same as ab? Check on this #############################
        value = wRAA + UZR + position + (20/600)*ab
        

In [65]:
calc = metric_calculator(data=df)

In [66]:
df['year'] = df['gameid'].apply(lambda x: int(x[3:7]))

In [67]:
athletics = list(set(
    list(set(df[(df['year'] == 2018) & (df['gameid'].str.contains('OAK')) & (df['battingteam'] == 1)]['batter'])) + 
    list(set(df[(df['year'] == 2018) & (df['vteam']=='OAK') & (df['battingteam'] == 0)]['batter']))
))

In [68]:
for player_id in athletics:
    print(player_id, calc.calculate(player_ids=[player_id], metric='BA'))

    gameid
0      0.0
1      0.0
2    510.0
3    260.0
4      8.0
5      0.0
6      3.0
7      0.0
8      1.0
9      8.0
10     2.0
11     1.0
12     1.0
13     1.0
14    71.0
15     3.0
16    25.0
17     0.0
18    11.0
19     3.0
20   142.0
21    57.0
22     4.0
23    41.0
24     0.0
25     0.0
canhm001 0.23921568627450981
    gameid
0      0.0
1      0.0
2    290.0
3    119.0
4      2.0
5      0.0
6      1.0
7      0.0
8      1.0
9      5.0
10     2.0
11     0.0
12     0.0
13     2.0
14    43.0
15     0.0
16    12.0
17     0.0
18     3.0
19     3.0
20    79.0
21    41.0
22     0.0
23    27.0
24     0.0
25     0.0
piscs001 0.2639138240574506
    gameid
0      0.0
1      0.0
2     70.0
3     36.0
4      2.0
5      0.0
6      0.0
7      0.0
8      0.0
9      1.0
10     2.0
11     0.0
12     0.0
13     0.0
14    20.0
15     1.0
16     5.0
17     0.0
18     2.0
19     0.0
20    32.0
21     9.0
22     3.0
23     1.0
24     0.0
25     0.0
martn003 0.29605263157894735
    gameid
0      0.0
1

In [319]:
calc.calculate(player_ids=['troum001'], metric='BA')

0.30671834625322997

In [320]:
calc.calculate(player_ids=['troum001'], metric='SLG')

0.5733850129198966

In [321]:
calc.calculate(player_ids=['troum001'], metric='wRAA')

ab 3870 bb 693 hbp 65 sf 44 hits 679 224 44 240


377.88579896685354